# Data refinement

In [1]:
# Imports
import pandas as pd


In [4]:
# Load data. Right now we are using backup file, as we haven't scraped the full dataset.
# Initially I load everything as str so I can change decimal etc. so typecasting goes without problems.
df = pd.read_csv("C:\\Users\\Teemu\\Desktop\\Housing_market_analysis\\Housing-market-analysis-with-webscraped-data\\Jul-10-2022-house-market-data.csv", dtype={'Type': 'str', 'Address': 'str', 'District': 'str', 'City': 'str', 'Price (€)': 'str', 'Area (sqm.)': 'str', 'Year built': 'str', 'Form of ownership': 'str'}, index_col=0)
df.head()

,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership
0,"Kerrostalo | 1h, kk, kph.",Soukanlahdenkatu 9,Armonkallio,Tampere,169000,28,1963,Omistusasunto
1,Kerrostalo | 1h + kk + alk + kh + p,Lepolankatu 3 A,Kivistönmäki,Lahti,39000,34,1972,Omistusasunto
2,Luhtitalo | 1h + kk,Viljatullintie 4,Järvenperä,Espoo,139000,"28,5",2018,Omistusasunto
3,Kerrostalo | 2h + avokeittiö + kph / wc + vh,Maariankatu 2,Keskusta,Turku,215000,48,1962,Omistusasunto
4,"Omakotitalo | 4h, k, aula, vh, th, s-tilat, wc...",Puunkaatajantie 23b,Heikinlaakso,Helsinki,215000,90,1933,Omistusasunto


In [5]:
# Lets edit some of the types
print('Types initially:\n\n', df.dtypes)

# First we will drop all rows with undefined values in price or area. This is okay as the dataset is quite large and we can not get the values from anywhere.
df.drop(df.index[df['Price (€)'] == '-'], inplace=True)
df.drop(df.index[df['Area (sqm.)'] == '-'], inplace=True)

# Then we will replace decimal from ',' to '.' and delete whitespace (\xa0) from Area column (This is already done for price in webscraper) so we can typecast the objects to floats.
df['Price (€)'] = [x.replace(',', '.') for x in df['Price (€)']]
df['Price (€)'] = df['Price (€)'].astype('float')

df['Area (sqm.)'] = [x.replace(',', '.') for x in df['Area (sqm.)']]
df['Area (sqm.)'] = [x.replace('\xa0', '') for x in df['Area (sqm.)']]
df['Area (sqm.)'] = df['Area (sqm.)'].astype('float')

print('\nTypes after cleaning:\n\n', df.dtypes)


Types initially:

 Type                 object
Address              object
District             object
City                 object
Price (€)            object
Area (sqm.)          object
Year built           object
Form of ownership    object
dtype: object

Types after cleaning:

 Type                  object
Address               object
District              object
City                  object
Price (€)            float64
Area (sqm.)          float64
Year built            object
Form of ownership     object
dtype: object


In [6]:
# Lets edit the Type so that we get the type of the house (omakotitalo, rivitalo, etc.) and the room abbreviations to separate columns.
df[['Type', 'Rooms']] = df['Type'].str.split('|', 1, expand=True)
df.head()

,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership,Rooms
0,Kerrostalo,Soukanlahdenkatu 9,Armonkallio,Tampere,169000.0,28.0,1963,Omistusasunto,"1h, kk, kph."
1,Kerrostalo,Lepolankatu 3 A,Kivistönmäki,Lahti,39000.0,34.0,1972,Omistusasunto,1h + kk + alk + kh + p
2,Luhtitalo,Viljatullintie 4,Järvenperä,Espoo,139000.0,28.5,2018,Omistusasunto,1h + kk
3,Kerrostalo,Maariankatu 2,Keskusta,Turku,215000.0,48.0,1962,Omistusasunto,2h + avokeittiö + kph / wc + vh
4,Omakotitalo,Puunkaatajantie 23b,Heikinlaakso,Helsinki,215000.0,90.0,1933,Omistusasunto,"4h, k, aula, vh, th, s-tilat, wc, kuisti"


In [77]:
#Now that the types and structure is as wanted, lets work with empty values.
print(df.isnull().sum())

#As our type error handling in webscraper inserts None if error is risen, we count Nones in all columns.
type_none = len(df[df.Type == 'None'])
address_none = len(df[df.Address == 'None'])
district_none = len(df[df.District == 'None'])
price_none = len(df[df['Price (€)'] == 'None'])
area_none = len(df[df['Area (sqm.)'] == 'None'])
ownership_none = len(df[df['Form of ownership'] == 'None'])

print('\nNones in Type:',type_none, '\nNones in Address:', address_none, '\nNones in District:', district_none, '\nNones in Price:', price_none, '\nNones in Area:', area_none,'\nNones in Ownership:', ownership_none)

df

Type                   0
Address                0
District               0
City                   0
Price (€)              0
Area (sqm.)            0
Year built             0
Form of ownership      0
Rooms                169
dtype: int64

Nones in Type: 0 
Nones in Address: 0 
Nones in District: 3314 
Nones in Price: 0 
Nones in Area: 0 
Nones in Ownership: 0


,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership,Rooms
0,Kerrostalo,Soukanlahdenkatu 9,Armonkallio,Tampere,169000.0,28,1963,Omistusasunto,"1h, kk, kph."
1,Kerrostalo,Lepolankatu 3 A,Kivistönmäki,Lahti,39000.0,34,1972,Omistusasunto,1h + kk + alk + kh + p
2,Luhtitalo,Viljatullintie 4,Järvenperä,Espoo,139000.0,28.5,2018,Omistusasunto,1h + kk
3,Kerrostalo,Maariankatu 2,Keskusta,Turku,215000.0,48,1962,Omistusasunto,2h + avokeittiö + kph / wc + vh
4,Omakotitalo,Puunkaatajantie 23b,Heikinlaakso,Helsinki,215000.0,90,1933,Omistusasunto,"4h, k, aula, vh, th, s-tilat, wc, kuisti"
...,...,...,...,...,...,...,...,...,...
45948,Omakotitalo,"Teuvantie, Kesti",Kesti,Kurikka,115000.0,100,-,Omistusasunto,4h + k + ph + WC + vh + ullakko
45949,Erillistalo,Kirjosillankatu,Myllypohja,Lahti,268000.0,245,1991,Omistusasunto,"5h + k, sauna ja 2h + k parveke, + työtila j..."
45950,Rivitalo,Koparatie 3-5,Keskusta,Sodankylä,108500.0,80,1986,Omistusasunto,3h + k + s + v + katettu terassi
45951,Omakotitalo,Kaislarannantie5,None,Savonlinna,695000.0,175,2008,Omistusasunto,4mh + oh + k + s + khh + aula + 3wc + 2vh


We can notice that we have 3K+ Nones in District. This could be solved with Address and City. However, for now we'll delete these as this is just a hobby project. It might be solved later.

In [7]:
df.drop(df.index[df['District'] == 'None'], inplace=True)
df

,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership,Rooms
0,Kerrostalo,Soukanlahdenkatu 9,Armonkallio,Tampere,169000.0,28.0,1963,Omistusasunto,"1h, kk, kph."
1,Kerrostalo,Lepolankatu 3 A,Kivistönmäki,Lahti,39000.0,34.0,1972,Omistusasunto,1h + kk + alk + kh + p
2,Luhtitalo,Viljatullintie 4,Järvenperä,Espoo,139000.0,28.5,2018,Omistusasunto,1h + kk
3,Kerrostalo,Maariankatu 2,Keskusta,Turku,215000.0,48.0,1962,Omistusasunto,2h + avokeittiö + kph / wc + vh
4,Omakotitalo,Puunkaatajantie 23b,Heikinlaakso,Helsinki,215000.0,90.0,1933,Omistusasunto,"4h, k, aula, vh, th, s-tilat, wc, kuisti"
...,...,...,...,...,...,...,...,...,...
45947,Omakotitalo,Tikkalantie 159,Tikkala,Tohmajärvi,278000.0,225.0,1926,Omistusasunto,"4mh, wc, kylpyhuone, keittiö, sali"
45948,Omakotitalo,"Teuvantie, Kesti",Kesti,Kurikka,115000.0,100.0,-,Omistusasunto,4h + k + ph + WC + vh + ullakko
45949,Erillistalo,Kirjosillankatu,Myllypohja,Lahti,268000.0,245.0,1991,Omistusasunto,"5h + k, sauna ja 2h + k parveke, + työtila j..."
45950,Rivitalo,Koparatie 3-5,Keskusta,Sodankylä,108500.0,80.0,1986,Omistusasunto,3h + k + s + v + katettu terassi


In [9]:
# Lets remove all rows that have '-' in any of the following columns: Type, District, City, Price, Area, Year built or Form of Ownership
df.drop(df.index[df['Type'] == '-'], inplace=True)
df.drop(df.index[df['District'] == '-'], inplace=True)
df.drop(df.index[df['City'] == '-'], inplace=True)
df.drop(df.index[df['Price (€)'] == '-'], inplace=True)
df.drop(df.index[df['Area (sqm.)'] == '-'], inplace=True)
df.drop(df.index[df['Year built'] == '-'], inplace=True)
df.drop(df.index[df['Form of ownership'] == '-'], inplace=True)
df

,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership,Rooms
0,Kerrostalo,Soukanlahdenkatu 9,Armonkallio,Tampere,169000.0,28.0,1963,Omistusasunto,"1h, kk, kph."
1,Kerrostalo,Lepolankatu 3 A,Kivistönmäki,Lahti,39000.0,34.0,1972,Omistusasunto,1h + kk + alk + kh + p
2,Luhtitalo,Viljatullintie 4,Järvenperä,Espoo,139000.0,28.5,2018,Omistusasunto,1h + kk
3,Kerrostalo,Maariankatu 2,Keskusta,Turku,215000.0,48.0,1962,Omistusasunto,2h + avokeittiö + kph / wc + vh
4,Omakotitalo,Puunkaatajantie 23b,Heikinlaakso,Helsinki,215000.0,90.0,1933,Omistusasunto,"4h, k, aula, vh, th, s-tilat, wc, kuisti"
...,...,...,...,...,...,...,...,...,...
45946,Kerrostalo,Luhtikatu 3 F 18,Liipola,Lahti,79500.0,53.5,1972,Omistusasunto,2h + k + vh
45947,Omakotitalo,Tikkalantie 159,Tikkala,Tohmajärvi,278000.0,225.0,1926,Omistusasunto,"4mh, wc, kylpyhuone, keittiö, sali"
45949,Erillistalo,Kirjosillankatu,Myllypohja,Lahti,268000.0,245.0,1991,Omistusasunto,"5h + k, sauna ja 2h + k parveke, + työtila j..."
45950,Rivitalo,Koparatie 3-5,Keskusta,Sodankylä,108500.0,80.0,1986,Omistusasunto,3h + k + s + v + katettu terassi


In [10]:
# Lets save the refined data.

filename = 'REFINED-Jul-10-2022-house-market-data.csv'
df.to_csv(filename, encoding="utf-8-sig")